In [1]:
#[15] Certidão Negativa de Protesto (PG)

In [2]:
#Bibliotecas:

#Deprecated:
#from selenium.webdriver import Chrome

#New Version: Selenium 4.0 Beta 1
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
 
#Options:
from selenium.webdriver.chrome.options import Options

#chrome_options.add_argument("--disable-extensions")
#chrome_options.add_argument('-headless')

from selenium.webdriver.support.ui import WebDriverWait

#Exceptions:
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException

from random import randrange       #random number (used in secs to wait - time)
import time                        #wait to run
import getpass                     #usr info
import pandas as pd
import json
import os
import glob
import shutil

In [3]:
#Configurações Impressão em .pdf:
caminho_download = 'C:\\Users\\felip\\Downloads'
chrome_options = Options()
settings = {
       "recentDestinations": [{
            "id": "Save as PDF",
            "origin": "local",
            "account": "",
        }],
        "selectedDestinationId": "Save as PDF",
        "version": 2
    }
exclude_default_swithes = ["disable-client-side-phishing-detection",
    "safebrowsing-disable-auto-update",
    "safebrowsing-disable-download-protection",
    "ignore-certificate-errors",]
prefs = {"plugins.always_open_pdf_externally": True,
        "download.default_directory": caminho_download,       
        "printing.print_preview_sticky_settings.appState": json.dumps(settings),
        "savefile.default_directory": 'C:\\Users\\felip\\Downloads'}     #ADD
chrome_options.add_experimental_option("prefs",prefs)
chrome_options.add_argument('--kiosk-printing') #ADD
chrome_options.add_experimental_option("excludeSwitches", exclude_default_swithes)
chrome_options.add_experimental_option("useAutomationExtension", False)

In [4]:
#Cria uma pasta para a SC caso ela já não exista no sistema:
def criaPastaSC(codigo_SC): #Captura o código da SC
    newpath = r'C:\\Users\\felip\\OneDrive\\Área de Trabalho\\Documentacao\\Certidoes' 
    newpath = newpath + '\\' + codigo_SC
    if not os.path.exists(newpath):
        os.makedirs(newpath)
        print("Pasta:" + newpath + "criada.")
    print("PAsta já existe.")

In [5]:
def moveCert(codigo_SC):
#Move a certidão baixada da pasta Downloads para a pasta da SC:

    #Lista os arquivos da Pasta Downloads:
    list_of_files = glob.glob(r'C:\Users\felip\Downloads\*.pdf') # * means all if need specific format then *.csv
    print(list_of_files)
    
    #Captura o último arquivo:
    latest_file = max(list_of_files, key=os.path.getctime)

    #Move o arquivo à pasta da SC:
    destino = "C:\\Users\\felip\\OneDrive\\Área de Trabalho\\Documentacao\\Certidoes\\"
    destino = destino + str(codigo_SC)
    shutil.move(latest_file, destino)
    print(destino)

In [6]:
#Captura o nome do usuário:
usr = getpass.getuser()

In [7]:
#INPUTS:
df_cpf = pd.read_excel("C:/Users/felip/OneDrive/Área de Trabalho/Documentacao/Robos/INPUT/A_Consultar.xlsx", engine='openpyxl')

In [8]:
#Acesso ao Site:

#Link para Certidão Negativa de Protesto (PG):
url = "http://www.cartorioshoji.com.br/Home/Servico?Identificador=3"

#Varre cada CLiente/Pedido do Input:
for index, row in df_cpf.iterrows():

    #Identifica se a certidão conseguiu ser extraída atribuindo True:
    sucesso = False
        
    while sucesso != True:
        
        print("Realizando SC" + str(index))
        
        #Adicionar condicional para capturar da base o tipo de documento utilizado na busca do site:
        #(CPF, RG ou CNPJ)
        
        #Dados do Cliente:
        num_cpf = str(df_cpf['CPF'][index])
        
        #Inicializa o Chromedriver:
        #s = Service('chromedriver.exe')
        driver = webdriver.Chrome("chromedriver.exe", options=chrome_options)
        
        #Página Alvo:
        driver.get(url)
        
        #Clica na janela de utilização de cookies - botão Aceito Continuar:
        #time.sleep(3)
        #elemento = (By.XPATH, "/html[@class='js no-touch history boxshadow csstransforms3d csstransitions video svg webkit chrome win js sticky-header-enabled sticky-header-active']/body/div[@id='mensagemInferiorDeCookie']/div[@class='aceite-cookie-bottom']/button[@class='btn btn-orion']")
        #acesso = driver.find_element(*elemento).click()
                
        #Preenche o inputbox Número do documento:
        time.sleep(2)
        elemento = (By.ID, "txtNrDoctoTiProt")
        acesso = driver.find_element(*elemento).send_keys(num_cpf)
        
        #Clica no Botão Consultar:
        time.sleep(3)
        elemento = (By.ID, "bt_consultaTiProt_consultar")
        acesso = driver.find_element(*elemento).click()
        
        #Clica no Botão Imprimir:
        time.sleep(3)
        #elemento = (By.XPATH, "/html[@class='js no-touch history boxshadow csstransforms3d csstransitions video svg webkit chrome win js sticky-header-enabled sticky-header-active']/body[@class='modal-open']/div[@class='container ServicosContainer']/div[@class='row']/div[@class='col']/div[@id='card']/div[@class='row']/div[@id='pnPopUpModalTitulosProtestados']/div[@class='modal-dialog']/div[@class='modal-content']/div[@class='modal-footer']/a[@id='bt_consultatiprot_imprimir']")
        #acesso = driver.find_element(*elemento).click()   
        #elemento = (By.name, "bt_consultatiprot_imprimir")
        #acesso = driver.find_element(*elemento).click()
        #elemento = (By.name, "bt_consultatiprot_imprimir")
        #elemento = (By.CLASS_NAME, "input[id = 'bt_consultatiprot_imprimir']")
        #acesso = driver.find_element(*elemento).click()
        
        #Da Scrolldown para carregar toda a página:
        driver.execute_script("window.scrollTo(1080, document.body.scrollHeight);")
        elemento = (By.CSS_SELECTOR, "a[onclick*='ImprimirConteudoPainelConsultaProtesto")
        acesso = driver.find_element(*elemento).click()
                    
        #Cria pasta na rede para SC caso ela ainda não exista:
        nm_pasta = str(index) + "_rb8"
        criaPastaSC(str(index))
        
        #Move Certidão para a pasta da SC:
        time.sleep(2)
        #Pausa para que o Download seja completo antes de mover e renomear o arquivo da certidão:
        moveCert(str(index))
        sucesso = True  

Realizando SC0


C:\Users\felip\AppData\Local\Temp/ipykernel_52512/1181355980.py:24: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe", options=chrome_options)


ElementClickInterceptedException: Message: element click intercepted: Element <a onclick="ImprimirConteudoPainelConsultaProtesto(impressao_consulta_protestos);" class="btn btn-orion" id="bt_consultatiprot_imprimir" name="bt_consultatiprot_imprimir">...</a> is not clickable at point (630, 537). Other element would receive the click: <p>...</p>
  (Session info: chrome=96.0.4664.110)
Stacktrace:
Backtrace:
	Ordinal0 [0x008A6903+2517251]
	Ordinal0 [0x0083F8E1+2095329]
	Ordinal0 [0x00742848+1058888]
	Ordinal0 [0x00773509+1258761]
	Ordinal0 [0x00771868+1251432]
	Ordinal0 [0x0076F65D+1242717]
	Ordinal0 [0x0076E4A8+1238184]
	Ordinal0 [0x00764037+1196087]
	Ordinal0 [0x007864D3+1336531]
	Ordinal0 [0x00763A36+1194550]
	Ordinal0 [0x007865BA+1336762]
	Ordinal0 [0x00795BBF+1399743]
	Ordinal0 [0x0078639B+1336219]
	Ordinal0 [0x007627A7+1189799]
	Ordinal0 [0x00763609+1193481]
	GetHandleVerifier [0x00A35904+1577972]
	GetHandleVerifier [0x00AE0B97+2279047]
	GetHandleVerifier [0x00936D09+534521]
	GetHandl